## Generate images for evaluation

In [1]:
import os,json
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

from inference import *


# inital args
args = parse_args(["--pretrained_model_name_or_path", "../../logs/face/e4t/000001/30", "--num_images_per_prompt", "4", "--scheduler_type", "ddim", \
                   "--image_path_or_url", "../../pcs_dataset/face/000001/face.jpg", \
                   "--num_inference_steps", "50", "--guidance_scale", "7.5"])


/home/huwentao/anaconda3/envs/e4t/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Single Subject Generation
single_subject = []                            # "backpack"
# Single Prompt Generation
single_prompt = []                            # e.g. ["a {0} {1} near the pool"]

num_generation = 4

output_path = "../../outputs/face/e4t"
subjects = os.listdir("../../logs/face/e4t/")
dataset_info_path = "../../pcs_dataset/info.json"

if len(single_subject):
    subjects = single_subject

for subject in subjects:
    
    print(f"***** Subject: {subject} *****")

    os.makedirs(os.path.join(output_path, subject), exist_ok=True)

    if len(single_prompt):
        prompts = single_prompt
    else:
        with open(dataset_info_path, 'r') as file:
          data_info = json.load(file)["face"]
        prompts = data_info["prompt_accessory"] + data_info["prompt_context"] + data_info["prompt_action"] + data_info["prompt_style"]

    for prompt in prompts:
        prompt = prompt.replace("a {0} {1}", "*s")
        print(f"**Prompt**: {prompt}")

        if prompt not in os.listdir(os.path.join(output_path, subject)):
            args.prompt = prompt
            args.image_path_or_url = f"../../pcs_dataset/face/{subject}/face.jpg"
            args.pretrained_model_name_or_path = f"../../logs/face/e4t/{subject}/30"

            images = e4t_inference(args)

            save_path = os.path.join(output_path, subject, f"{prompt}")
            os.makedirs(save_path, exist_ok=True)

            for img_idx, img in enumerate(images):
                img.save(os.path.join(save_path, f"{img_idx:04d}.jpg"))

## Evaluation

### Calculate the similarity for each sample
Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [1]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
import numpy as np
from PIL import Image
from utils.clip_eval import evaluate_t2i
from utils.face_eval import calculate_identity_similarity

/home/huwentao/anaconda3/envs/fastcomposer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data
    
def add_evaluation(file_path, new_data):
    # check whether the file exist
    if not os.path.exists(file_path):
        with open(file_path, 'w') as file:
            json.dump(convert_to_native(new_data), file, indent=4)
    else:
        with open(file_path, 'r') as file:
            data = json.load(file)
        data.update(convert_to_native(new_data))
        with open(file_path, 'w') as file:
            json.dump(data, file, indent=4)

In [ ]:
outputs_path = "../../outputs/face/e4t"
eval_res_path = "../../eval_results/face/e4t"
dataset_path = "../../pcs_dataset/face"
dataset_info_path = "../../pcs_dataset/info.json"

with open(dataset_info_path, 'r') as file:
    data_info = json.load(file)["face"]

id_with_gender = data_info["id_with_gender"]

os.makedirs(eval_res_path, exist_ok=True)

id_list = os.listdir(outputs_path)

for id in id_list:
    evaluation_res = dict()
    print(f"***** Face ID: {id} *****")

    res_for_each_subject =dict()

    ref_image = Image.open(os.path.join(dataset_path, id, "face.jpg")).convert("RGB")

    for prompt in os.listdir(os.path.join(outputs_path, id)):
        prompt_eval = prompt.replace("*s", f"a {id_with_gender[id]}")
        print(f"**Prompt**: {prompt_eval}")

        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, id, prompt)):
            generate_img_path = os.path.join(outputs_path, id, prompt, generate_img_name)
            
            text_similarity = evaluate_t2i(generate_img_path, prompt_eval)

            enerated_image = Image.open(generate_img_path).convert("RGB")
            identity_similarity = float(calculate_identity_similarity(device=torch.device("cuda"), generated_image=enerated_image, ref_image=ref_image))

            res_for_each_prompt[generate_img_name] = [identity_similarity, text_similarity]
        
        res_for_each_subject[prompt] = res_for_each_prompt
        print(res_for_each_prompt)

    evaluation_res[id] = res_for_each_subject

    add_evaluation(os.path.join(eval_res_path, "evaluation_results.json"), evaluation_res)

### Calculate the average similarity

In [4]:
import json

eval_res_path = "../../eval_results/face/e4t/evaluation_results.json"

with open(eval_res_path, "r") as f:
    data = json.load(f)

img_sim = 0.0
text_sim = 0.0
cnt = 0

for subject in data:
    for prompt in data[subject]:
        for sample in data[subject][prompt]:
            img_sim = img_sim + data[subject][prompt][sample][0]
            text_sim = text_sim + data[subject][prompt][sample][1]
            cnt = cnt + 1
print("Identity Similarity: ", img_sim/cnt, "\nText Similarity:", text_sim/cnt)

Identity Similarity:  0.17530545070942025 
Text Similarity: 0.26860020955403646
